In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import plotly_express as px
from ast import literal_eval
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import HDBSCAN

SEED = 42

## Read data
Training and testing projects and measurements are created in the /data/Data_Ingestion.ipynb notebook

In [2]:
train_projects = pd.read_csv("../../data/raw_data/train_projects.csv").set_index('project_code')
test_projects = pd.read_csv("../../data/raw_data/test_projects.csv").set_index('project_code')
projects = pd.concat([train_projects, test_projects])
projects.shape

(3178, 12)

## Create measurements frame for full data
* Still removing projects with incorrectly formatted band
* Still removing projects with > 26 measurements

In [3]:
train_measurements = pd.read_csv('../../../train_measurements.csv').set_index('project_code')
test_measurements = pd.read_csv('../../../test_measurements.csv').set_index('project_code')
measurements = pd.concat([train_measurements, test_measurements])
measurements.head()

,project_title,project_abstract,fs_type,low_freq,high_freq,science_category,science_keyword,band,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,,,,,,,,
2016.1.01288.S,The molecular gas properties of radio-AGN at z...,Energetic feedback from AGN is believed to pla...,line,137.06,138.93,Active galaxies,"Outflows, jets, feedback",4.0,1,1.87,137.995,The molecular gas properties of radio-AGN at z...,the molecular gas properties of radio agn at z...,molecular gas properties radio agn z energetic...,the molecular gas property of radio agn at z e...,molecular gas property radio agn z energetic f...
2016.1.01288.S,The molecular gas properties of radio-AGN at z...,Energetic feedback from AGN is believed to pla...,line,138.87,140.73,Active galaxies,"Outflows, jets, feedback",4.0,1,1.86,139.800,The molecular gas properties of radio-AGN at z...,the molecular gas properties of radio agn at z...,molecular gas properties radio agn z energetic...,the molecular gas property of radio agn at z e...,molecular gas property radio agn z energetic f...
2016.1.01288.S,The molecular gas properties of radio-AGN at z...,Energetic feedback from AGN is believed to pla...,line,149.17,151.03,Active galaxies,"Outflows, jets, feedback",4.0,1,1.86,150.100,The molecular gas properties of radio-AGN at z...,the molecular gas properties of radio agn at z...,molecular gas properties radio agn z energetic...,the molecular gas property of radio agn at z e...,molecular gas property radio agn z energetic f...
2016.1.01288.S,The molecular gas properties of radio-AGN at z...,Energetic feedback from AGN is believed to pla...,line,150.97,152.84,Active galaxies,"Outflows, jets, feedback",4.0,1,1.87,151.905,The molecular gas properties of radio-AGN at z...,the molecular gas properties of radio agn at z...,molecular gas properties radio agn z energetic...,the molecular gas property of radio agn at z e...,molecular gas property radio agn z energetic f...
2016.1.01288.S,The molecular gas properties of radio-AGN at z...,Energetic feedback from AGN is believed to pla...,line,143.87,145.74,Active galaxies,"Outflows, jets, feedback",4.0,1,1.87,144.805,The molecular gas properties of radio-AGN at z...,the molecular gas properties of radio agn at z...,molecular gas properties radio agn z energetic...,the molecular gas property of radio agn at z e...,molecular gas property radio agn z energetic f...


## LDA Model

### LDA class

In [4]:
class LDA_Model:
    def __init__(self, N_topics=50):
        self.N_topics = N_topics
        self.countVectorizer = CountVectorizer()
        self.lda = LatentDirichletAllocation(n_components=self.N_topics, random_state=SEED)
    
    def fit(self, corpus):
        termFrequency = self.countVectorizer.fit_transform(corpus)
        self.lda.fit(termFrequency)
        return self.lda.transform(termFrequency)

    # Additional method to transform new data
    def transform(self, corpus):
        termFrequency = self.countVectorizer.transform(corpus)
        return self.lda.transform(termFrequency)

#### Initialize Model

In [5]:
lda_model = LDA_Model(N_topics=50)

#### Fit model on training set

In [6]:
topics = lda_model.fit(projects.lemmatized_no_sw_text)

In [7]:
words = lda_model.countVectorizer.get_feature_names_out()

### Inspect top words for topics to see if they are salient
We can also use these later in user-facing tools for transparency

In [60]:
import pandas as pd

N = 10  # number of top words to show
topic_components = lda_model.lda.components_

# Create an empty list to store words by topic
topic_word_list = []

for topic_idx, topic in enumerate(topic_components):
    print(f"Topic {topic_idx}:")
    # Get the indices of the top N words for this topic
    top_word_indices = topic.argsort()[-N:][::-1]
    # Populate the list with top words and their weights
    word_num = 0
    for word_idx in top_word_indices:
        word = words[word_idx]
        weight = topic[word_idx]
        print(f"{word} (weight: {weight:.2f})")
        topic_word_list.append({'Topic': topic_idx, 'Word_Number':word_num, 'Word': word, 'Weight': weight})
        word_num += 1

# Create a DataFrame from the list
topic_words = pd.DataFrame(topic_word_list).set_index(['Topic', 'Word_Number'])

Topic 0:
star (weight: 1387.11)
formation (weight: 1046.84)
cloud (weight: 734.49)
molecular (weight: 689.62)
cluster (weight: 647.60)
gas (weight: 611.21)
galaxy (weight: 544.15)
dense (weight: 479.05)
region (weight: 363.39)
form (weight: 359.08)
Topic 1:
hole (weight: 245.82)
black (weight: 230.24)
sgr (weight: 108.33)
supermassive (weight: 88.60)
observation (weight: 58.10)
galactic (weight: 50.80)
scale (weight: 44.70)
horizon (weight: 31.53)
image (weight: 29.12)
provide (weight: 26.73)
Topic 2:
gas (weight: 662.92)
molecular (weight: 448.81)
galaxy (weight: 440.16)
agn (weight: 365.33)
dense (weight: 237.59)
starburst (weight: 181.99)
study (weight: 163.69)
nuclear (weight: 132.18)
observation (weight: 120.81)
hco (weight: 120.51)
Topic 3:
filament (weight: 504.99)
shock (weight: 300.08)
cloud (weight: 136.95)
gas (weight: 129.57)
molecular (weight: 125.90)
dense (weight: 102.96)
filamentary (weight: 97.98)
formation (weight: 93.51)
sio (weight: 89.44)
structure (weight: 79.65)


In [61]:
topic_words

Word       Weight
Topic Word_Number                        
0     0                 star  1387.112921
      1            formation  1046.844243
      2                cloud   734.491779
      3            molecular   689.620294
      4              cluster   647.604234
...                      ...          ...
49    5                 form   153.517156
      6               specie   150.824777
      7               region   150.164410
      8              organic   147.030448
      9                grain   124.138520

[500 rows x 2 columns]

### Inspect training document-topic data frames

In [9]:
doc_topic = pd.DataFrame(topics)
doc_topic = doc_topic.set_index(projects.index.values)
doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
2016.1.01288.S,0.000392,0.000392,0.210363,0.000392,0.000392,0.000392,0.000392,0.000392,0.029965,0.000392,...,0.000392,0.000392,0.000392,0.000392,0.000392,0.000392,0.462452,0.000392,0.000392,0.000392
2018.1.01077.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194
2018.1.00437.S,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.329201,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192
2021.1.00637.S,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,...,0.000222,0.000222,0.000222,0.989111,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222
2012.1.00786.S,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.381998,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171


### Group documents to highest matching topic

### Take highest matching topic for each project

In [10]:
doc_topic['max_topic'] = doc_topic.apply(lambda x: x.argmax(), axis=1)

In [11]:
doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,max_topic
2016.1.01288.S,0.000392,0.000392,0.210363,0.000392,0.000392,0.000392,0.000392,0.000392,0.029965,0.000392,...,0.000392,0.000392,0.000392,0.000392,0.000392,0.462452,0.000392,0.000392,0.000392,46
2018.1.01077.S,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,...,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,0.000194,21
2018.1.00437.S,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,...,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,0.000192,33
2021.1.00637.S,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,...,0.000222,0.000222,0.989111,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,43
2012.1.00786.S,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.381998,0.000171,0.000171,0.000171,...,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,0.000171,14


### Add `max_topic` to `projects`

In [12]:
projects = projects.join(doc_topic['max_topic'].to_frame())

In [13]:
projects.max_topic.value_counts().to_frame().sort_index()

,count
max_topic,
0,342
1,14
2,128
3,36
4,29
5,21
6,58
7,32
8,28


### Inspect some topic stats

In [14]:
projects.max_topic.value_counts().describe()

count     50.000000
mean      63.560000
std       67.128923
min       10.000000
25%       18.500000
50%       34.500000
75%       85.250000
max      342.000000
Name: count, dtype: float64

There are a few topics that match to a large number of documents. Perhaps we need a better topic model or to group documents by project_topic vector similarity.

### Eyeball comparison of documents by max topic
Check that these are similar projects, given they are in the same topic

In [15]:
with pd.option_context('display.max_colwidth', None):
    for proj in projects.query('max_topic == 0').index[:5]:
        print(f'project {proj} combined title and abstract: {projects.loc[proj].raw_text}')

project 2013.1.00833.S combined title and abstract: Using CI to Map the Real Structure of a Low-Metallicity Starburst. Low-metallicity environments frequently exhibit starburst behavior, forming most of their stars in massive clusters. The physical conditions in the molecular ISM must dictate this distinct mode of star formation. Unfortunately, our typical tracer of the molecular ISM -- CO emission -- is confined to the densest parts of molecular clouds. Instead, much of the molecular gas is colocated with atomic carbon. We propose using CI to characterize molecular gas using ALMA's superior resolution (both spatial and spectral) and surface brightness sensitivity to map the bulk of the molecular clouds in the nearby low-metallicity starburst NGC 5253. In this low metallicity environment, we expect a larger fraction of the molecular gas to be traced by the CI instead of the CO. With these data, we will (1) measure the properties of molecular clouds in a low metallicity starburst and (2

### Add `max_topic` to `measurements` frame to be able to group measurements by max topic

In [16]:
measurements = measurements.join(projects.max_topic.to_frame())
measurements.head()

,project_title,project_abstract,fs_type,low_freq,high_freq,science_category,science_keyword,band,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text,max_topic
project_code,,,,,,,,,,,,,,,,,
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",3.0,1,0.24,90.500,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...,2
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",3.0,1,0.23,90.815,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...,2
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",3.0,1,0.23,91.805,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...,2
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",3.0,1,0.23,93.005,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...,2
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,Active galaxies,"Active Galactic Nuclei (AGN)/Quasars (QSO), Me...",6.0,1,0.94,218.060,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...,2


### Generate projects measurements
We want projects with a list of all of the relevant measurement data

**NOTE!!!**
You should not sort these, however tempting. We need to preserve the relationships of the entries to not lose measurement information.

In [17]:
project_measurements = measurements.groupby(measurements.index)\
    .agg({
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist()
    })
project_measurements.head()

,low_freq,high_freq,med_freq,diff_freq
project_code,,,,
2011.0.00010.S,"[90.38, 90.7, 91.69, 92.89, 217.59, 218.67, 21...","[90.62, 90.93, 91.92, 93.12, 218.53, 219.6, 21...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9..."
2011.0.00017.S,"[87.72, 89.54, 99.72, 101.54, 91.37, 93.19, 10...","[89.6, 91.42, 101.59, 103.42, 93.24, 95.07, 10...","[88.66, 90.48, 100.655, 102.48, 92.305, 94.13,...","[1.88, 1.88, 1.87, 1.88, 1.87, 1.88, 1.88, 1.8..."
2011.0.00028.S,"[342.36, 344.24, 354.28, 355.79, 342.36]","[344.23, 346.11, 356.16, 357.66, 344.24]","[343.295, 345.175, 355.22, 356.725, 343.3]","[1.87, 1.87, 1.88, 1.87, 1.88]"
2011.0.00039.S,"[338.32, 340.18, 350.37, 352.2]","[340.2, 342.06, 352.24, 354.08]","[339.26, 341.12, 351.305, 353.14]","[1.88, 1.88, 1.87, 1.88]"
2011.0.00046.S,"[99.65, 101.48, 111.52, 113.36]","[101.52, 103.36, 113.4, 115.23]","[100.585, 102.42, 112.46, 114.295]","[1.87, 1.88, 1.88, 1.87]"


### Create `topic_freqs`
* Dataframe of all measurements within a topic used for HDBSCAN
* This is an intermediate dataframe as it is a convenient format to loop through and run HDBSCAN
* We make something very similar to this later but in a more usable format

In [18]:
topic_freqs = measurements\
    .reset_index()\
    .groupby('max_topic')\
    .agg({
        'project_code': lambda x: x.tolist(), 
        'low_freq': lambda x: round(x, 4).tolist(),
        'high_freq': lambda x: round(x, 4).tolist(),
        'med_freq': lambda x: round(x, 4).tolist(),
        'diff_freq': lambda x: round(x, 4).tolist(),
        'band': lambda x: x.astype('int64').tolist()
    })
topic_freqs.head()

,project_code,low_freq,high_freq,med_freq,diff_freq,band
max_topic,,,,,,
0,"[2011.0.00039.S, 2011.0.00039.S, 2011.0.00039....","[338.32, 340.18, 350.37, 352.2, 86.27, 88.15, ...","[340.2, 342.06, 352.24, 354.08, 88.14, 90.03, ...","[339.26, 341.12, 351.305, 353.14, 87.205, 89.0...","[1.88, 1.88, 1.87, 1.88, 1.87, 1.88, 1.87, 1.8...","[7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, ..."
1,"[2016.1.01154.V, 2016.1.01154.V, 2016.1.01154....","[212.17, 214.17, 226.17, 228.17, 212.14, 214.1...","[214.04, 216.04, 228.04, 230.04, 214.01, 216.0...","[213.105, 215.105, 227.105, 229.105, 213.075, ...","[1.87, 1.87, 1.87, 1.87, 1.87, 1.87, 1.86, 1.8...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 6, ..."
2,"[2011.0.00010.S, 2011.0.00010.S, 2011.0.00010....","[90.38, 90.7, 91.69, 92.89, 217.59, 218.67, 21...","[90.62, 90.93, 91.92, 93.12, 218.53, 219.6, 21...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9...","[3, 3, 3, 3, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, ..."
3,"[2012.1.00812.S, 2012.1.00812.S, 2012.1.00812....","[85.47, 87.01, 96.15, 97.86, 224.72, 226.83, 8...","[87.34, 87.95, 98.03, 99.73, 226.6, 228.7, 85....","[86.405, 87.48, 97.09, 98.795, 225.66, 227.765...","[1.87, 0.94, 1.88, 1.87, 1.88, 1.87, 1.88, 1.8...","[3, 3, 3, 3, 6, 6, 3, 3, 6, 6, 6, 6, 6, 6, 6, ..."
4,"[2012.1.00320.S, 2012.1.00320.S, 2012.1.00320....","[218.67, 219.21, 231.21, 233.61, 219.19, 219.9...","[220.66, 221.2, 233.2, 235.6, 220.19, 220.93, ...","[219.665, 220.205, 232.205, 234.605, 219.69, 2...","[1.99, 1.99, 1.99, 1.99, 1.0, 1.0, 1.0, 1.0, 1...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, ..."


## Cluster cleaning function

How do we handle > 2 band overlaps?

* Currently we just roll with it.
    
* Should call out which topic, and cluster

* Alternatively, throw error <- this is hard because it completely stops any training

In [19]:
# Code to check for clusters that span at least two bands
def cluster_cleaning(topic_meas_df, min_cluster_size):
    dummy_label = 100000  # Used to make new labels and ensure we're adding new clusters. Cluster labels will be reset eventually

    for clst in np.unique(topic_meas_df.cluster_label):
        # HDBSCAN labels noise as -1 so we skip this row for cleaning
        if clst != -1:
            # Subset topic measurement dataframe to current cluster
            clst_subset = topic_meas_df[topic_meas_df.cluster_label == clst].sort_values('med_freq')
            
            # Extract band information for cluster subset
            band = np.unique(clst_subset.band)

            # If there are multiple bands in measurements for cluster we want to break them up
            if band.size > 1:
                # Loop over bands in cluster and make a dataframe for each band
                bnd_dict = [clst_subset[clst_subset.band == bnd] for bnd in band]

                # Loop over cluster-band frames
                for df in bnd_dict:
                # Check if number of measurements in this band and cluster is > min_cluster_size
                # If it is less, simply assign those measurements back to noise, since we don't want too small clusters
                # Otherwise, there are enough measurments to keeping "this" part of the cluster, so make a new cluster for it 
                    if df.shape[0] < min_cluster_size:
                        topic_meas_df.loc[df.index, 'cluster_label'] = -1
                    else:
                        topic_meas_df.loc[df.index, 'cluster_label'] = dummy_label
                        dummy_label += 1
                        
    # Re-label clusters to be a continuous range from -1 to N
    new_labels = [n-1 for n in range(len(np.unique(topic_meas_df.cluster_label)))]
    label_counter = 0   # Used to increment through new_labels
    
    # Loop over cluster labels and update them
    for reclust in np.unique(topic_meas_df.cluster_label):
        # Set cluster label to new_labels
        topic_meas_df.loc[topic_meas_df.cluster_label == reclust, 'cluster_label'] = new_labels[label_counter]
        label_counter += 1

## HDBSCAN Code
### Loop over topics and run HDBSCAN

In [20]:
topic_cluster_widths = []          # List of cluster widths by topic to ensure generated clusters are not too wide (list of lists)
total_num_clusters = 0             # List of number of clusters for each topic
topic_cluster_list = []            # List of dataframes with clusters by topic. Used to make a main topic-cluster data frame later
topic_measurement_list = []        # List of dataframes with measurements by topic. Used to make a main topic-measurement data frame later
min_cluster_size = 5               # Minimum number of points to quantify "dense" for HDBSCAN. Also used in cluster cleaning

# Loop over topics
for tpc in set(projects.max_topic.values):
    # Fit HDBSCAN for each topic
    # Note that these can be parameterized for each of the topics generated
    # Note one can give HDBSCAN a max_cluster_size parameter to ensure clusters do not grow too large
    db = HDBSCAN(min_cluster_size=min_cluster_size)\
        .fit(list(zip(topic_freqs.loc[tpc].med_freq)))
    
    # Get labels from HDBSCAN
    labels = db.labels_

    # Create topic-measurement dataframe
    # This has all of the measurements for all of the projects in an LDA topic
    topic_measurement_temp = pd.DataFrame.from_dict({'low_freq':topic_freqs.loc[tpc].low_freq,
                                                     'med_freq':topic_freqs.loc[tpc].med_freq,
                                                     'high_freq':topic_freqs.loc[tpc].high_freq,
                                                     'band':topic_freqs.loc[tpc].band,
                                                     'project_code':topic_freqs.loc[tpc].project_code,
                                                     'cluster_label':labels})
    
    # Format topic_measurement_temp and add to list for final frame later
    topic_measurement_temp = pd.concat({tpc: topic_measurement_temp}, names=['topic'])
    topic_measurement_temp.index.names = ['topic', 'measurement']
    topic_measurement_list.append(topic_measurement_temp)
    
    # Clean clusters in topic_measurment, breaking up clusters that span more than one band
    cluster_cleaning(topic_measurement_temp, min_cluster_size)

    # Generate topic_cluster_temp data frame for this topic
    # Clusters are defined by the minimum and maximum median_freq for all labeled measurements
    topic_cluster_temp = topic_measurement_temp.groupby('cluster_label').agg(
        med_freq=('med_freq', 'median'),
        min_freq=('med_freq', 'min'),
        max_freq=('med_freq', 'max'),
        count_meas=('med_freq', 'count'),
        count_proj=('project_code', 'nunique'),
        min_band=('band', 'min'),
        max_band=('band', 'max'),
        mode_band=('band', 'mean')
    )

    # Stat aggregation
    n_projects = topic_cluster_temp.count_proj.sum()
    n_measurements = topic_cluster_temp.count_meas.sum()
    if -1 in topic_cluster_temp.index:
        n_noise = topic_cluster_temp.loc[-1].count_meas
        noise_proportion = n_noise/n_measurements
    else:
        n_noise = 0
        noise_proportion = 0
    signal_proportion = 1-noise_proportion

    # Stat callouts
    print(f'HDBSCAN Results for topic {tpc}')
    print(f'Number of projects in topic: {n_projects}')
    print(f'Total number of measurements: {n_measurements}')
    print(f'Estimated number of noise measurements: {n_noise}')
    print(f'Noise proportion: {round(noise_proportion, 3)}')
    print(f'Signal proportion: {round(signal_proportion, 3)}\n')

    # Sort index
    topic_cluster_temp = topic_cluster_temp.sort_index()

    # Add width for cleaned clusters
    topic_cluster_temp['width'] = topic_cluster_temp.max_freq - topic_cluster_temp.min_freq

    # Add topic index to topic_cluster_temp
    topic_cluster_temp = pd.concat({tpc: topic_cluster_temp}, names=['topic'])
    topic_cluster_temp.index.names = ['topic', 'cluster']

    # Append topic_cluster_temp to topic_cluster_stats for analysis later
    topic_cluster_list.append(topic_cluster_temp)

    # Print width stats for topic clusters excluding noise
    print('Topic width stats excluding noise')
    print(topic_cluster_temp.drop(-1, axis=0, level=1).width.describe())

    # Add number of clusters for topic to total_num_clusters
    total_num_clusters += (len(list(topic_cluster_temp.index.values))-1)

    # Print cluster data frame with relevant columns for tuning
    print('')
    print(f'Cluster data frame for topic {tpc}')
    with pd.option_context('display.max_rows', None):
        print(topic_cluster_temp[['min_freq', 'max_freq', 'count_meas', 'mode_band', 'width']]\
          .sort_values(['width', 'min_freq'], ascending=False))
    print('=========================================\n')

print(f'Total number of clusters across topics: {total_num_clusters}')

HDBSCAN Results for topic 0
Number of projects in topic: 1686
Total number of measurements: 2729
Estimated number of noise measurements: 412.0
Noise proportion: 0.151
Signal proportion: 0.849

Topic width stats excluding noise
count    241.000000
mean       0.540436
std        1.849026
min        0.000000
25%        0.025000
50%        0.080000
75%        0.260000
max       17.710000
Name: width, dtype: float64

Cluster data frame for topic 0
               min_freq  max_freq  count_meas  mode_band    width
topic cluster                                                    
0     -1         84.080   468.970         412   4.990291  384.890
       1        805.960   823.670           8  10.000000   17.710
       0        681.345   694.520          11   9.000000   13.175
       5        150.510   161.960          17   4.000000   11.450
       6        139.065   146.280          14   4.000000    7.215
       19       288.140   295.010          17   7.000000    6.870
       4        866.245  

### Build full topic-measurement data frame

In [21]:
topic_measurement = pd.concat(topic_measurement_list)

In [22]:
topic_measurement.loc[0]

,low_freq,med_freq,high_freq,band,project_code,cluster_label
measurement,,,,,,
0,338.32,339.260,340.20,7,2011.0.00039.S,63
1,340.18,341.120,342.06,7,2011.0.00039.S,-1
2,350.37,351.305,352.24,7,2011.0.00039.S,93
3,352.20,353.140,354.08,7,2011.0.00039.S,-1
4,86.27,87.205,88.14,3,2011.0.00217.S,91
...,...,...,...,...,...,...
2724,217.88,218.000,218.12,6,2023.1.01720.S,179
2725,220.28,220.400,220.52,6,2023.1.01720.S,193
2726,220.88,221.000,221.12,6,2023.1.01720.S,-1


### Visualization of HDBSCAN on Topics
Choose a topic in `inspect_topic` and let it rip! It uses the `topic_measurement_frame` to generate the images. There's a little bit of extra processing though, so we create a helper dataframe, `inspect_topic_frame` to make sure everything runs smoothly.

* This should probably become a function, at least the plotting part
* Maybe check out plotly "strip" charts

In [23]:
inspect_topic = 0
inspect_topic_frame = topic_measurement.loc[inspect_topic]

inspect_topic_frame = inspect_topic_frame.sort_values('cluster_label', ascending=False)
inspect_topic_frame.cluster_label = inspect_topic_frame.cluster_label.astype('str')

# Add noise binary column for plot symbol
inspect_topic_frame['noise'] = np.where(inspect_topic_frame.cluster_label == '-1', 1, 0)

# Noise and Signal
itf_noise = inspect_topic_frame.noise.sum()
itf_signal = inspect_topic_frame.shape[0] - itf_noise

# Set symbols for plot
# We set all points to be 'circle' using the px number 0, and then change noise to 'x'
symbols = list(np.zeros(np.unique(inspect_topic_frame.cluster_label).shape[0], 'int'))
symbols[-1] = 'x'

# Create plot
fig = px.scatter(inspect_topic_frame,
                 x='med_freq',
                 y='cluster_label',
                 color='cluster_label',
                 symbol='cluster_label',
                 symbol_sequence=symbols,
                 title=f"HDBSCAN Generated Clusters for Topic {inspect_topic} <br><sup>{itf_signal} Clustered Measurements with {itf_noise} Noise Measurements</sup>",
                 labels={
                     'med_freq':'Median Frequency (GHz)',
                     'index':'Index',
                     'cluster_label':'Cluster Label'
                 })
fig.update_traces(marker={'size': 15, 'opacity':0.5})

fig.show()

In [24]:
topic_measurement.loc[25]

,low_freq,med_freq,high_freq,band,project_code,cluster_label
measurement,,,,,,
0,288.96,289.900,290.84,7,2011.0.00064.S,150
1,290.79,291.730,292.67,7,2011.0.00064.S,143
2,300.84,301.775,302.71,7,2011.0.00064.S,176
3,302.71,303.650,304.59,7,2011.0.00064.S,180
4,288.94,289.880,290.82,7,2011.0.00064.S,150
...,...,...,...,...,...,...
2160,410.99,411.930,412.87,8,2023.1.01354.S,21
2161,213.54,214.005,214.47,6,2023.1.01362.S,77
2162,214.51,215.445,216.38,6,2023.1.01362.S,-1


### Build full topic-cluster data frame

## **THIS IS A VERY IMPORTANT DATAFRAME IT THIS IS THE CORE RESULT OF THE MINING APPROACH!!!!!!**

This data frame holds all of the cluster info for each of the generated topics

* Pretty much all of the cluster stats in the code cell above can be derived from this

In [25]:
topic_cluster = pd.concat(topic_cluster_list)

## Check for clusters with 0 width
These are clusters that all have a single measurement

In [26]:
topic_cluster[topic_cluster.width == 0]

med_freq  min_freq  max_freq  count_meas  count_proj  min_band  \
topic cluster                                                                   
0     113        93.175    93.175    93.175           8           6         3   
      114        93.170    93.170    93.170          10          10         3   
      118        97.900    97.900    97.900           5           5         3   
      126        97.910    97.910    97.910           5           4         3   
      127        97.905    97.905    97.905           8           7         3   
...                 ...       ...       ...         ...         ...       ...   
47    88        219.560   219.560   219.560          15          15         6   
      89        219.555   219.555   219.555          10          10         6   
      90        230.535   230.535   230.535          12          11         6   
      91        230.530   230.530   230.530          10          10         6   
      95        220.390   220.390   220.390           9           9         6   

               max_band  mode_band  width  
topic cluster                              
0     113             3        3.0    0.0  
      114             3        3.0    0.0  
      118             3        3.0    0.0  
      126             3        3.0    0.0  
      127             3        3.0    0.0  
...                 ...        ...    ...  
47    88              6        6.0    0.0  
      89              6        6.0    0.0  
      90              6        6.0    0.0  
      91              6        6.0    0.0  
      95              6        6.0    0.0  

[81 rows x 9 columns]

### We want to be able to click these eventually, so anything with width 0 give a width of 0.005

In [27]:
topic_cluster.loc[topic_cluster.width == 0, 'width'] = 0.005

In [28]:
topic_cluster.loc[0].sort_values('count_proj', ascending=False)

,med_freq,min_freq,max_freq,count_meas,count_proj,min_band,max_band,mode_band,width
cluster,,,,,,,,,
-1,219.3525,84.080,468.970,412,171,3,8,4.990291,384.890
33,102.6525,102.295,103.000,40,21,3,3,3.000000,0.705
169,345.4750,345.150,345.830,32,18,7,7,7.000000,0.680
41,98.9400,98.825,99.005,28,17,3,3,3.000000,0.180
105,87.8550,87.630,88.080,20,15,3,3,3.000000,0.450
...,...,...,...,...,...,...,...,...,...
200,229.8550,229.805,229.890,5,2,6,6,6.000000,0.085
158,233.4250,233.405,233.495,5,2,6,6,6.000000,0.090
35,113.3450,113.330,113.355,5,2,3,3,3.000000,0.025


In [29]:
topic_cluster.sample(10)

med_freq  min_freq  max_freq  count_meas  count_proj  min_band  \
topic cluster                                                                   
41    0        650.9400   625.010   666.890          17           5         9   
32    74       262.0700   262.070   262.075           5           4         6   
14    0        690.3125   675.695   705.020          16           1         9   
40    20       339.4725   339.425   339.520           8           6         7   
0     142      232.3650   232.290   232.370           5           3         6   
43    59       249.5550   247.350   251.360          34          21         6   
25    173      300.1625   299.765   301.135           6           6         7   
      129      144.1025   143.935   144.315           6           5         4   
27    8        214.4850   214.480   214.520           7           2         6   
18    16        98.7825    97.990    99.705          14           9         3   

               max_band  mode_band   width  
topic cluster                               
41    0               9        9.0  41.880  
32    74              6        6.0   0.005  
14    0               9        9.0  29.325  
40    20              7        7.0   0.095  
0     142             6        6.0   0.080  
43    59              6        6.0   4.010  
25    173             7        7.0   1.370  
      129             4        4.0   0.380  
27    8               6        6.0   0.040  
18    16              3        3.0   1.715

### Compute cluster signal to noise proportions

In [30]:
sn_list = []
for clst in np.unique(topic_cluster.index.get_level_values(0)):
    clst_sig = np.sum(topic_cluster.loc[topic_cluster.index.get_level_values(1) != -1].loc[clst].count_meas)
    clst_noise = np.sum(topic_cluster.loc[topic_cluster.index.get_level_values(1) == -1].loc[clst].count_meas)
    sn_list.append({'signal':clst_sig, 'noise':clst_noise})

In [31]:
signal_noise_frame = pd.DataFrame(sn_list)
signal_noise_frame.index.name = 'cluster'
signal_noise_frame['signal_prop'] = (signal_noise_frame.signal)/(signal_noise_frame.signal + signal_noise_frame.noise)
signal_noise_frame['noise_prop'] = (signal_noise_frame.noise)/(signal_noise_frame.signal + signal_noise_frame.noise)
signal_noise_frame

,signal,noise,signal_prop,noise_prop
cluster,,,,
0,2317,412,0.849029,0.150971
1,69,8,0.896104,0.103896
2,800,176,0.819672,0.180328
3,247,35,0.875887,0.124113
4,112,29,0.794326,0.205674
5,102,12,0.894737,0.105263
6,413,65,0.864017,0.135983
7,269,39,0.873377,0.126623
8,178,39,0.820276,0.179724


In [32]:
signal_noise_frame.describe()

,signal,noise,signal_prop,noise_prop
count,50.000000,50.000000,50.000000,50.000000
mean,394.580000,75.060000,0.852813,0.147187
std,425.507672,86.502427,0.056358,0.056358
min,66.000000,1.000000,0.679739,0.013699
25%,128.000000,17.250000,0.821371,0.113901
50%,214.500000,37.500000,0.849689,0.150311
75%,574.250000,98.750000,0.886099,0.178629
max,2317.000000,412.000000,0.986301,0.320261


## Inspect topic-clusters

In [33]:
topic_cluster.describe()

,med_freq,min_freq,max_freq,count_meas,count_proj,min_band,max_band,mode_band,width
count,2097.000000,2097.000000,2097.000000,2097.000000,2097.000000,2097.000000,2097.000000,2097.000000,2097.000000
mean,234.032258,230.218748,240.761118,11.197902,6.135908,5.385312,5.484025,5.433997,10.542563
std,124.788963,125.109195,132.335317,17.328286,7.587916,1.742578,1.747915,1.713530,59.564392
min,39.575000,36.080000,43.105000,1.000000,1.000000,1.000000,1.000000,1.000000,0.005000
25%,114.672500,112.360000,115.080000,6.000000,3.000000,3.000000,3.000000,3.000000,0.135000
50%,228.830000,225.975000,230.010000,8.000000,5.000000,6.000000,6.000000,6.000000,0.600000
75%,291.300000,289.205000,311.835000,12.000000,7.000000,7.000000,7.000000,7.000000,2.065000
max,896.135000,893.615000,906.795000,412.000000,171.000000,10.000000,10.000000,10.000000,775.050000


In [34]:
topic_cluster.width.describe()

count    2097.000000
mean       10.542563
std        59.564392
min         0.005000
25%         0.135000
50%         0.600000
75%         2.065000
max       775.050000
Name: width, dtype: float64

In [35]:
topic_cluster.query('width > 10 and cluster != -1')\
    .sort_values(['count_meas', 'width'], ascending=False)\
    .head(50)


,,med_freq,min_freq,max_freq,count_meas,count_proj,min_band,max_band,mode_band,width
topic,cluster,,,,,,,,,
30,3,102.7525,86.040,113.375,58,11,3,3,3.0,27.335
43,62,146.5625,132.080,162.005,54,13,4,4,4.0,29.925
31,2,257.2100,244.155,267.970,31,4,6,6,6.0,23.815
7,4,186.8675,165.480,200.850,28,4,5,5,5.0,35.370
35,1,98.1150,85.025,113.230,27,4,3,3,3.0,28.205
14,2,97.7900,86.840,106.130,27,5,3,3,3.0,19.290
45,1,333.6550,310.255,371.755,24,5,7,7,7.0,61.500
43,21,464.8650,461.050,472.905,22,8,8,8,8.0,11.855
30,2,146.7150,129.365,158.870,21,6,4,4,4.0,29.505


### Check to see there are no clusters spanning bands

In [36]:
topic_cluster.query('max_band - min_band > 1 and cluster != -1')\
    .sort_values(['width', 'count_meas'], ascending=False)\
    .head(30)

,,med_freq,min_freq,max_freq,count_meas,count_proj,min_band,max_band,mode_band,width
topic,cluster,,,,,,,,,


### Inspect an individual topic's clusters

In [37]:
topic_cluster.loc[0]

,med_freq,min_freq,max_freq,count_meas,count_proj,min_band,max_band,mode_band,width
cluster,,,,,,,,,
-1,219.3525,84.080,468.970,412,171,3,8,4.990291,384.890
0,690.8950,681.345,694.520,11,3,9,9,9.000000,13.175
1,814.8350,805.960,823.670,8,2,10,10,10.000000,17.710
2,130.8250,128.640,133.120,5,3,4,4,4.000000,4.480
3,853.2325,850.245,856.215,8,2,10,10,10.000000,5.970
...,...,...,...,...,...,...,...,...,...
236,230.4575,230.455,230.465,6,4,6,6,6.000000,0.010
237,230.3650,230.365,230.365,5,5,6,6,6.000000,0.005
238,230.3600,230.360,230.360,8,8,6,6,6.000000,0.005


### Histogram of topic clusters

**Compare this to the scatter plot above. These two charts in tandem are good. Maybe we can combine them somehow**

Hover info needs work

In [38]:
import plotly.graph_objects as go
def plot_topic_clusters(tc_frame:pd.DataFrame, topic:int):
    figure = go.Figure()
    figure.add_trace(
        go.Bar(
            x=tc_frame.query(f'topic== {topic} and cluster != -1').med_freq,
            y=tc_frame.query(f'topic== {topic} and cluster != -1').count_meas,
            #name=dict(color=tc_frame.query(f'topic== {topic} and cluster != -1').index.get_level_values(level=1)),
            width=tc_frame.query(f'topic== {topic} and cluster != -1').width.to_list()
            # hoverinfo=(
            #     tc_frame.query(f'topic== {topic} and cluster != -1').min_freq,
            #     tc_frame.query(f'topic== {topic} and cluster != -1').max_freq
            # )
        )
    )
    figure.update_layout(
    title=(f'Areas of Interest for Topic {topic}'),
    xaxis_title='Frequency (GHz)',
    yaxis_title='Count of Measurements',
    )
    figure.show()

In [39]:
plot_topic_clusters(topic_cluster, 25)

## Save relevant dataframes

In [62]:
topic_words.to_csv('../../data/model_outputs/topic_words.csv')
topic_cluster.to_csv('../../data/model_outputs/topic_cluster.csv')
topic_measurement.to_csv('../../data/model_outputs/topic_measurement.csv')